<H3> HM APIs testing </H3>

In [2]:
'''HM internal API tests'''

import requests
import json

base_url = "https://api.hotelmap.com"
headers = {
    "Content-Type": "application/json",
    "Authorization": "Bearer hm_live_lI%Jesi1%JJpinzU^eKzVmPp502D7cfPTK13ChNm&05dRFHrnpNPXQP0XJVU7S7xg3EjmER9FN1"
}

def upcoming_events(year):
    '''
    Upcoming events for a given year
    Args: A year

    API Reference:
    GET /v1/cheesefactory/eventfeeds/{year}
    '''
    endpoint = "/v1/cheesefactory/eventfeeds/" + str(year)
    url = base_url + endpoint
    payload = {}
    
    return requests.get(url, headers=headers, json=payload).json()

    
def venue_search(venue, city, top=0):
    """
    Search for venues by name and city.
    Args:
        venue (str): Name or keyword
        city (str): 
        top (int): select only the top result
        
    API Reference:
    POST /v1/cheesefactory/venues/search
    """
    endpoint = "/v1/cheesefactory/venues/search"
    url = base_url + endpoint
    payload = {
        "search_term": venue,  # keyword or venue name
        "city": city           # city to narrow the search
    }

    # If 'top' is specified, return the first venue found
    if top:
        return requests.post(url, headers=headers, json=payload).json()['data']['venues'][0]
    else:
        return requests.post(url, headers=headers, json=payload).json()


def venue_data(venueid):
    """
    Does what it says    
    Args:
        venueid (str): HM ID
        
    API Reference:
    GET /v1/cheesefactory/venues/{venue_id}
    """
    endpoint = f"/v1/cheesefactory/venues/{venueid}"
    url = base_url + endpoint
    payload = {}  # GET request usually doesn't need payload
    return requests.get(url, headers=headers, json=payload)


def load_json(filename):
    with open(filename, "r") as f:
        data = json.load(f)

    print(type(data))  # Print type for debugging (could be dict or list)
    return data


def save_result(response, filename):
    with open(filename, "w") as f:
        json.dump(response, f, indent=4)  # Pretty-print with indentation
    
    return response

In [26]:
'''Return a list of upcoming events in HM system as a spreadsheet'''

import pandas as pd
from IPython.display import display

year = 2028
events = upcoming_events(year)

In [16]:
len(events)

20

In [27]:

rows = []
for event in events:
    e = event.get('event', {})
    rows.append({
        "title": e.get('title'),
        "start_date": e.get('date_start'),
        "end_date": e.get('date_end'),
        "venue_id": e.get('venue_id'),
        "is_venue_id_hotel": e.get('is_venue_id_hotel'),
        "city": e.get("city"),
        "map_key": e.get("map_key"),
        "latitude": e.get("latitude"),
        "longitude": e.get("longitude")
    })

df = pd.DataFrame(rows)

display(df)
df.to_excel(f"data/{str(year)}_events_hotels.xlsx", index=False)

res = save_result(events, f"data/{str(year)}_events.json")

,title,start_date,end_date,venue_id,is_venue_id_hotel,city,map_key,latitude,longitude
0,(DRAFT) RW Baird Convention,2028-06-11,2028-06-16,483322,NO,Milwaukee,MACV8,43.040099,-87.917473
1,64th Annual Meeting - EASD2028,2028-09-18,2028-09-20,483207,NO,København S,M8QAL,55.638011,12.578839
2,GBTA Convention 2028,2028-07-24,2028-07-26,482126,NO,Philadelphia,MAXWC,39.954910,-75.160347
3,TBA,2028-10-28,2028-11-01,35404,NO,Paris,M45BX,48.832769,2.286309
4,GBTA VDR Europe Conference,2028-11-06,2028-11-10,481915,NO,Zürich,MLNVU,47.411716,8.539943
5,International Conference on Isotopes,2028-02-01,2028-02-29,479792,NO,Baden,MJYR6,47.478672,8.305396
6,International Conference on Isotopes,2028-02-01,2028-02-29,18301,NO,Zürich,MSL6M,47.366559,8.536162
7,EADV Congress,2028-09-13,2028-09-16,35404,NO,Paris,MD5JQ,48.832769,2.286309
8,World Cup of Hockey 2028 (Eishockey),2028-02-05,2028-02-17,483012,NO,Düsseldorf,MYYXD,51.220319,6.792765
9,World Cup of Hockey 2028,2028-02-05,2028-02-17,483012,NO,Düsseldorf,M3CGH,51.220319,6.792765


In [18]:
res

{'id': 209789,
 'event': {'map_key': 'MO21D',
  'affiliate_id': 'OVY6SYQ6P6',
  'title': 'EASL LCS 2027',
  'sub_title': 'Palacio de Congresos ',
  'venue_id': 17441,
  'has_venue_id': 'YES',
  'is_venue_id_hotel': 'NO',
  'city': 'Madrid',
  'country': 'Spain',
  'country_code': 'ES',
  'latitude': 40.418944406378,
  'longitude': -3.692125082016,
  'embed_count': 0,
  'status': 'future',
  'event_period_in_days': 2,
  'event_attendees': 0,
  'date_start': '2027-01-14',
  'date_end': '2027-01-16',
  'eventwebsite': None,
  'created': '2025-03-03 00:00:00',
  'event_owner': None,
  'event_owner_email': None,
  'event_recipient_field': '',
  'pro_url': None,
  'pro_overview': 'NOT_CREATED',
  'private_rates_id': 'NOT_CREATED'},
 'company': {'company_name': 'Congrex Switzerland Demo',
  'account_type': 'SAAS',
  'owners': None,
  'owners_email': None,
  'contact_name': '',
  'contact_email': '',
  'contact_firstname': '',
  'contact_lastname': '',
  'contact_type': 'NOT_AVAILABLE',
  'saa